In [0]:
import requests
import pandas as pd
import json
from pathlib import Path

# import eia_api.json
with open("eia_api.json", "r") as f:
    config = json.load(f)

# ---- 설정 ----
API_KEY = config["eia_api"]

url = "https://api.eia.gov/v2/petroleum/pri/spt/data/"
params = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data[0]": "value",
    "facets[product][]": "EPJK",  # Kerosene-type jet fuel
    "start": "2005-01",
    "end": "2025-05",
    "sort[0][column]": "period",
    "sort[0][direction]": "asc",   # ascending order for time series
    "offset": 0,
    "length": 5000
}

r = requests.get(url, params=params, timeout=30)
js = r.json()

# Extract into DataFrame
df = pd.DataFrame(js["response"]["data"])
df["date"] = pd.to_datetime(df["period"])
df = df[["date", "value"]].rename(columns={"value": "jet_fuel_usd_per_gal"})
df = df.sort_values("date").set_index("date")

print(df.head(), "\n", df.tail())
# df.to_csv("jet_fuel_price_2005_2025.csv", encoding="utf-8-sig")

OUTDIR = Path("./data")
OUTDIR.mkdir(parents=True, exist_ok=True)

df.to_csv("data/jet_fuel_price_2005_2025.csv", encoding="utf-8-sig")
print("Saved to data/jet_fuel_price_2005_2025.csv")
